In [2]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import scipy.stats as stats

from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu

import statsmodels.api as sm
from statsmodels.formula.api import ols

In [17]:
# Statistical analysis of the effect of NO (SNP treatment) on high CO2 induced stomatal closure

sheetname1 = 'Fig_7A' # Figure 7A - effect of NO (SNP treatment) on high CO2 induced stomatal closure

#below are data of other figures. The code for analysis may need adjustments (e.g. updating headers) to work on those data.
#sheetname1 = 'Fig_S1' # Figure S1 - effect of SNAP treatment on high CO2 induced stomatal closure
#sheetname1 = 'Fig_7B' # SNP can induce stomatal closure under ambient CO2
#sheetname1 = 'Fig_8A' # cPTIO (NO scavenger) can impair high CO2 induced stomatal closure
#sheetname1 = 'Fig_8D' # NO inhibits the phosphatase activity of ABI2 

#df = pd.read_excel('CO2_model_data_20231120.xlsx', header=[0,1,2,3], sheet_name=sheetname1)
df = pd.read_excel('S7_Data. Summary of all experimental data 2024-0219.xlsx', header=[0,1,2,3], sheet_name=sheetname1)

# melting multiindex data
df1 = pd.melt(df, var_name=['Treatment','CO2','Time','Day'], value_name='aperture').dropna()
#df1.to_csv('CO2_model_data_20231120'+sheetname1+'_melted.csv',index=False, encoding='utf_8_sig')
df1

,Treatment,CO2,Time,Day,aperture
0,Preincubation,control(0 time),10min,Day1,2.841
1,Preincubation,control(0 time),10min,Day1,3.108
2,Preincubation,control(0 time),10min,Day1,2.728
3,Preincubation,control(0 time),10min,Day1,2.004
4,Preincubation,control(0 time),10min,Day1,1.201
...,...,...,...,...,...
2692,0.15mM SNP,800ppm,20min,Day3,2.043
2693,0.15mM SNP,800ppm,20min,Day3,2.246
2694,0.15mM SNP,800ppm,20min,Day3,1.287
2695,0.15mM SNP,800ppm,20min,Day3,1.724


In [18]:
# calculating means and SEMs for each day, to create the figures 
dfx=df1
# Mean
dfx1 = pd.pivot_table(dfx, index=['Treatment'], values=['aperture'], columns=['Time','CO2'])
print(dfx1)
# SEM
dfx2 = pd.pivot_table(dfx, index=['Treatment'], values=['aperture'], columns=['Time','CO2'], aggfunc='sem')
print(dfx2)

               aperture                                              
Time              10min                               20min          
CO2              400ppm    800ppm control(0 time)    400ppm    800ppm
Treatment                                                            
0.05mM SNP     2.502929  1.760190             NaN  2.529368  1.680803
0.15mM SNP     2.447195  1.653053             NaN  2.474158  1.581602
0mM SNP        2.679207  2.156947             NaN  2.621809  1.980833
Preincubation       NaN       NaN        2.731031       NaN       NaN
               aperture                                              
Time              10min                               20min          
CO2              400ppm    800ppm control(0 time)    400ppm    800ppm
Treatment                                                            
0.05mM SNP     0.059132  0.038014             NaN  0.069425  0.033805
0.15mM SNP     0.052334  0.035351             NaN  0.063584  0.030520
0mM SNP        0.063

In [20]:
# Student's t-test 
# Below are example tests for Fig.7A, 7B and 8A. To use these code on the data of other figures, please update the formating, e.g. the column names (headers)

#reformatting data
dfx1 = pd.pivot_table(df1, values=['aperture'], columns=['Treatment','Time','CO2','Day'])
dfx2 = pd.melt(dfx1, var_name=['Treatment','time','CO2','Day'], value_name='aperture').dropna() 
dfx2 = dfx2[dfx2['Treatment']!='Preincubation']

# H0: In Col-0 under high CO2 and 10 min/20 min, the mean aperture under 0.15mM SNP is similar to the mean aperture without SNP

s1 = dfx2[(dfx2['Treatment']=='0mM SNP')&(dfx2['time']=='10min')&(dfx2['CO2']=='800ppm')]['aperture']
s2 = dfx2[(dfx2['Treatment']=='0.15mM SNP')&(dfx2['time']=='10min')&(dfx2['CO2']=='800ppm')]['aperture']
print (ttest_ind(s1, s2,equal_var=False))

s1 = dfx2[(dfx2['Treatment']=='0mM SNP')&(dfx2['time']=='20min')&(dfx2['CO2']=='800ppm')]['aperture']
s2 = dfx2[(dfx2['Treatment']=='0.15mM SNP')&(dfx2['time']=='20min')&(dfx2['CO2']=='800ppm')]['aperture']
print (ttest_ind(s1, s2,equal_var=False))

# conclusion: reject H0, i.e. SNP causes significant change of aperture size

s1 = dfx2[(dfx2['Treatment']=='0mM SNP')&(dfx2['time']=='20min')&(dfx2['CO2']=='800ppm')]['aperture']
s2 = dfx2[(dfx2['Treatment']=='0.15mM SNP')&(dfx2['time']=='10min')&(dfx2['CO2']=='800ppm')]['aperture']
print (ttest_ind(s1, s2,equal_var=False))

Ttest_indResult(statistic=3.6503864256308067, pvalue=0.02773486510194757)
Ttest_indResult(statistic=4.6278833423478485, pvalue=0.03444178806061061)
Ttest_indResult(statistic=3.0201277793679178, pvalue=0.039677943152063834)


In [5]:
sheetname1 = 'Fig_7B' # SNP can induce stomatal closure under ambient CO2

df = pd.read_excel('S7_Data. Summary of all experimental data 2024-0219.xlsx', header=[0,1,2], sheet_name=sheetname1)

# melting multiindex data
df1 = pd.melt(df, var_name=['Treatment','CO2','Day'], value_name='aperture').dropna()
#reformatting data
dfx1 = pd.pivot_table(df1, values=['aperture'], columns=['Treatment','CO2','Day'])
dfx2 = pd.melt(dfx1, var_name=['Treatment','CO2','Day'], value_name='aperture').dropna() 
dfx2 = dfx2[dfx2['Treatment']!='Preincubation']

s1 = dfx2[(dfx2['Treatment']=='0mM SNP')&(dfx2['CO2']=='400ppm')]['aperture']
s2 = dfx2[(dfx2['Treatment']=='0.2mM SNP')&(dfx2['CO2']=='400ppm')]['aperture']
print (ttest_ind(s1, s2, equal_var=False))


Ttest_indResult(statistic=8.019149054345954, pvalue=0.001569984747451356)


In [6]:
sheetname1 = 'Fig_8A' # cPTIO (NO scavenger) can impair high CO2 induced stomatal closure

df = pd.read_excel('S7_Data. Summary of all experimental data 2024-0219.xlsx', header=[0,1,2], sheet_name=sheetname1)

# melting multiindex data
df1 = pd.melt(df, var_name=['Treatment','CO2','Day'], value_name='aperture').dropna()
#reformatting data
dfx1 = pd.pivot_table(df1, values=['aperture'], columns=['Treatment','CO2','Day'])
dfx2 = pd.melt(dfx1, var_name=['Treatment','CO2','Day'], value_name='aperture').dropna() 
dfx2 = dfx2[dfx2['Treatment']!='Preincubation']

s1 = dfx2[(dfx2['Treatment']=='0mM cPTIO')&(dfx2['CO2']=='800ppm')]['aperture']
s2 = dfx2[(dfx2['Treatment']=='0.2mM cPTIO')&(dfx2['CO2']=='800ppm')]['aperture']
print (ttest_ind(s1, s2, equal_var=False))

Ttest_indResult(statistic=-8.074872073749695, pvalue=0.0019090132343228235)


In [33]:
sheetname1 = 'Fig_8D' # cPTIO (NO scavenger) can impair high CO2 induced stomatal closure

df = pd.read_excel('S7_Data. Summary of all experimental data 2024-0219.xlsx', skiprows=[0],sheet_name=sheetname1)

ref_values = df.iloc[0, 1:7]
for i in range (1,8):
    conc = df.iloc[i,0]
    values = df.iloc[i, 1:7]
    print (conc,ttest_ind(ref_values, values, equal_var=False))

GST-ABI2 (28 nM)+ SNP (5 micro molar) Ttest_indResult(statistic=2.7072504791461163, pvalue=0.028478835422117675)
GST-ABI2 (28 nM)+ SNP (10 micro molar) Ttest_indResult(statistic=3.8064467449856223, pvalue=0.0044276077113649575)
GST-ABI2 (28 nM)+ SNP (20 micro molar) Ttest_indResult(statistic=3.418696896647688, pvalue=0.00746380796057148)
GST-ABI2 (28 nM)+ SNP (25 micro molar) Ttest_indResult(statistic=8.059841902246456, pvalue=0.0002636090816580943)
GST-ABI2 (28 nM)+ SNP (30 micro molar) Ttest_indResult(statistic=9.262972123595068, pvalue=7.519795464219619e-05)
GST-ABI2 (28 nM)+ SNP (45 micro molar) Ttest_indResult(statistic=7.354519983918147, pvalue=3.5843402677251056e-05)
GST-ABI2(28 nM)+ SNP (50 micro molar) Ttest_indResult(statistic=5.695165285353425, pvalue=0.00020180154670808713)
